# Notes

## Make sure to build your lego database using the `db_postgres` container.

- You need to attach to the container and then navigate in a terminal to the `scratch` folder.
- In the scratch folder you should see a `lego.sql` file.
- Run the following psql command in the scratch folder `psql -d "postgres://postgres:postgres1234@db/lego" -f lego.sql`

## Exploring the database with Adminer

You can use [http://localhost:8080](http://localhost:8080) to see the database and tables you have created. If you see them there, the below code should work.

In [1]:
!pip install polars adbc-driver-postgresql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 53.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 59.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 60.5 MB/s eta 0:00:0000:01


In [2]:
import polars as pl
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext # https://spark.apache.org/docs/1.6.1/sql-programming-guide.html
from os.path import join, abspath
import os

# Example of Connecting to Database with Spark

In [6]:
# Create the session
conf = (SparkConf()
    .set("spark.ui.port", "4041")
    .set('spark.executor.memory', '4G')
    .set('spark.driver.memory', '45G')
    .set('spark.driver.maxResultSize', '10G')
    .set('spark.jars', '/home/jovyan/scratch/postgresql-42.2.18.jar'))

# Create the context
# sc = pyspark.SparkContext(conf=conf)
# sqlContext = SQLContext(sc)
spark = SparkSession.builder \
    .appName('test') \
    .config(conf=conf) \
    .getOrCreate()

In [15]:
# spark.stop()
properties = {
    'driver': 'org.postgresql.Driver',
    'url': 'jdbc:postgresql://db:5432/lego',
    'user': 'postgres',
    'password': 'postgres1234',
    'dbtable': 'lego_parts',
}
lego_parts = spark.read \
    .format('jdbc') \
    .option('driver', properties['driver']) \
    .option('url', properties['url']) \
    .option('dbtable', properties['dbtable']) \
    .option('user', properties['user']) \
    .load()
lego_parts.limit(5).toPandas()

,part_num,name,part_cat_id
0,0687b1,Set 0687 Activity Booklet 1,17
1,0901,Baseplate 16 x 30 with Set 080 Yellow House Print,1
2,0902,Baseplate 16 x 24 with Set 080 Small White Hou...,1
3,0903,Baseplate 16 x 24 with Set 080 Red House Print,1
4,0904,Baseplate 16 x 24 with Set 080 Large White Hou...,1


## Move Spark DF to Spark Database

In [16]:
lego_parts.write.saveAsTable("lego_parts_spark")

# Example of Connecting to Database with Polars

In [10]:
# https://docs.pola.rs/user-guide/io/database
uri = "postgresql://postgres:postgres1234@db:5432/lego"
pl.read_database_uri(query="SELECT datname from pg_database", uri=uri, engine="adbc")

datname
str
"""postgres"""
"""lego"""
"""template1"""
"""template0"""


In [11]:
lego_parts_pl = pl.read_database_uri(query="SELECT * FROM lego_parts", uri=uri, engine="adbc")
lego_parts_pl

part_num,name,part_cat_id
str,str,i32
"""0687b1""","""Set 0687 Activity Booklet 1""",17
"""0901""","""Baseplate 16 x 30 with Set 080…",1
"""0902""","""Baseplate 16 x 24 with Set 080…",1
"""0903""","""Baseplate 16 x 24 with Set 080…",1
"""0904""","""Baseplate 16 x 24 with Set 080…",1
…,…,…
"""zbb013""","""Znap Connector 3 x 3 - 4 way B…",43
"""zbb014""","""Znap Connector 1 x 3 - 2 way A""",43
"""zbb015""","""Znap Beam 3, 1 Hole""",43


## Move Polars Dataframe to Spark Database

In [12]:
parts_spdf = spark.createDataFrame(lego_parts_pl.to_pandas())
parts_spdf.write.saveAsTable("lego_parts")

In [17]:
spark.sql("SHOW TABLES").toPandas()

,namespace,tableName,isTemporary
0,default,lego_parts,False
1,default,lego_parts_spark,False
